In [ ]:
#%pip install mediapipe

In [ ]:
import mediapipe as mp
from matplotlib import pyplot as plt
import random
import glob
from util import draw_landmarks_on_image

In [ ]:
# extract dataset from zip
import zipfile
with zipfile.ZipFile("american-sign-language-letters.v1i.coco.zip","r") as zip_ref:
    zip_ref.extractall("american-sign-language-letters.v1i.coco")

In [ ]:
# trained model from mediapipe for hand landmark detection
model_path = 'hand_landmarker.task'

In [ ]:
# objects needed to create landmark detectors
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

In [ ]:
# create hand_landmarker detector from image
def create_landmark_detector_img():
    # use this method either in conjuction with a 'with' statement
    # or use detector.close() after you are done detecting 
    # to prevent resource leaking
    options = HandLandmarkerOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.IMAGE)
    return HandLandmarker.create_from_options(options)

In [ ]:
def detect_landmark_img(image):
    with create_landmark_detector_img() as landmark_detector:
        detection_result = landmark_detector.detect(image)
    return detection_result

In [ ]:
# prepare from image
def get_random_image():
    filename = random.choice(glob.glob('./american-sign-language-letters.v1i.coco/train/*.jpg'))
    return mp.Image.create_from_file(filename)

# # Load the input image from a numpy array.
# mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_image)

In [ ]:
def detect_and_show_image(image):
    detection_result = detect_landmark_img(image)
                    # method in util.py
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    plt.axis('off')
    plt.imshow(annotated_image)


In [ ]:
detect_and_show_image(get_random_image())

In [ ]:
# create hand_landmarker from live stream
def create_landmark_detector_stream():
    # use this method either in conjuction with a 'with' statement
    # or use detector.close() after you are done detecting 
    # to prevent resource leaking
    def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
        print('hand landmarker result: {}'.format(result))

    
    # Create a hand landmarker instance with the live stream mode:
    options = HandLandmarkerOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.LIVE_STREAM,
        result_callback=print_result)
    
    return HandLandmarker.create_from_options(options)